# História das Olimpíadas  
_(créditos ao prof. Rafael Moreira)_

Após um ano de atraso por conta da pandemia de Covid-19, as atenções do mundo todo se voltaram para Tóquio, no Japão, para acompanhar mais uma edição das Olimpíadas.

No Brasil não foi diferente, e muitos se uniram para torcer por nossos atletas em diferentes competições, tanto em esportes onde o Brasil já possui tradição quanto em novos esportes.

Vamos aproveitar o clima para estudar um pouco das Olimpíadas! Utilizaremos um _dataset_ com 120 anos de dados históricos das Olimpíadas, cobrindo desde os jogos de Atenas 1896 até Rio 2016. 

Faça o download do _dataset_ em https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results e carregue o arquivo ```athlete_events.csv``` para um DataFrame utilizando Pandas. Aproveite para explorar seu DataFrame e se familiarizar com a sua estrutura. 

OBS: Fique à vontade para acrescentar mais células Python conforme necessário em qualquer etapa do exercício.

In [1]:
import pandas as pd
import numpy as np
import os

path = os.getcwd()
df = pd.read_csv(path + '/dados/athlete_events.csv')
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/home/godara/LetsCode/M03_Bibliotecas_e_APIs/dados/athlete_events.csv'

In [ ]:
# Vamos remover os competidores com o time desconhecido para que o ranking dos países seja mais preciso 
# e substituir os espaços vazios na coluna de medalhas para que seja mais fácil trabalhar com esses dados.
# Além disso, vamos eliminar a coluna 'Games' pois a informação contida nela já se encontra nas colunas 
# 'Year' e 'Season'
df = df[df['Team'] != 'Unknown']
df['Medal'].fillna('No medal', inplace= True)
df.drop(columns='Games')

In [ ]:
# Precisamos aplicar essa função para analisarmos, por exemplo, 'Brasil' e 'Brasil-2' como apenas um país.
import re
def padroniza_pais(pais):
  return re.sub(r'(^[a-zA-Z0-9_ ]*)([-,/])([a-zA-Z0-9_ ]*$)', r"\1", pais)

df['Team'] = df['Team'].apply(padroniza_pais)

In [ ]:
# Nos primeiros jogos, principalmente, haviam equipes representando clubes, associações, universidades, etc.
# Assim, como todos estão associados a um 'NOC', escolhemos, para cada NOC, a equipe com mais registros.

repr_noc = df.groupby(['NOC', 'Team']).size().sort_values(ascending=False).reset_index()
repr_noc = repr_noc[['NOC', 'Team']].drop_duplicates(['NOC']).to_dict()
repr_noc = {repr_noc['NOC'][i]: repr_noc['Team'][i] for i in repr_noc['NOC']}
repr_noc

In [ ]:
# Atualizamos a coluna 'Team' com o representa 'padrao' do 'NOC' correpondente
df['Team'] = df['NOC'].map(repr_noc)

## 1. O Brasil nas Olimpíadas

Vamos começar estudando o desempenho do nossos próprio país. Gere um DataFrame novo contendo apenas as informações sobre atletas brasileiros.

In [ ]:
df_br = df[df['Team'] == 'Brazil'].copy()

### Medalhistas

Vamos focar um pouco nos casos de sucesso do Brasil. Use o seu DataFrame anterior para filtrar apenas informações sobre **medalhistas** brasileiros. 

**DICA:** observe como a coluna ```Medal``` é representada quando o atleta não ganhou medalha.

In [ ]:
df_medalhistas_br = df_br[df_br['Medal'] != 'No medal']
df_medalhistas_br


### Verão vs Inverno

Você deve ter notado que temos duas categorias distintas de jogos olímpicos, representados pela estação: temos os jogos de verão e os jogos de inverno, que ocorrem de maneira intercalada.

Agora que já conhecemos os medalhistas brasileiros, resposta: quantos atletas brasileiros receberam medalha nos jogos de verão e quantos receberam nos jogos de inverno?

In [ ]:
df_medalhistas_br.groupby('Season').size()

In [ ]:
# Fiz esse teste para confirmar que o Brasil participou das olimpíadas de Inverno, mas não conquistou nenhuma medalha.
df_br[df_br['Season'] == 'Winter']['Medal'].value_counts()

Os jogos de verão são bem mais populares do que os jogos de inverno no Brasil. Portanto, deste ponto em diante iremos focar apenas nos jogos de verão. Descarte de seu DataFrame os dados dos jogos de inverno.



In [ ]:
# Como não sabemos se usaremos o 'df_br' novamento, optamos pela criação de um novo DataFrame.
df_br_verao = df_br[df_br['Season'] == 'Summer']

### Atletas do Brasil

Vamos conhecer um pouco melhor nossos atletas. Descubra a altura e peso médio de nossos medalhistas.

In [ ]:
altura_media, peso_medio = df_medalhistas_br[['Height', 'Weight']].mean()
print(f'Altura Média: {altura_media:.2f} cm.')
print(f'Peso Médio: {peso_medio:.2f} kg.')

Imaginamos que diferentes esportes podem beneficiar diferentes tipos físicos, certo? Então refaça a análise anterior, mas obtendo os valores médios **por esporte**.

In [ ]:
import matplotlib.pyplot as plt
medias_altura_peso_medalhistas = df_medalhistas_br.groupby('Sport')[['Height', 'Weight']].mean()
mapm = medias_altura_peso_medalhistas

plt.barh(mapm.index, mapm['Height'], label = 'Height(Altura)')
plt.barh(mapm.index, mapm['Weight'], label = 'Weight(Peso)')
plt.xlabel('Weight(kg) - Height(cm)')

plt.legend(bbox_to_anchor=(1.01, 1.0))
plt.title('Medias de Altura e Peso')
plt.show()


Será que os dados acima influenciaram no interesse geral dos atletas pelo esporte ou realmente impactaram no desempenho deles? Podemos tentar descobrir se há algum tipo de correlação.

Você ainda possui o dataframe original contendo todos os atletas brasileiros, incluindo os sem medalha? Obtenha os valores médios de peso e altura por esporte daquele dataframe e compare-o com os dos medalhistas. Há alguma diferença significativa em algum esporte?

In [ ]:
# Média de altura e peso de todos atletas brasileiros
medias_altura_peso_geral = df_br_verao.groupby('Sport')[['Height', 'Weight']].mean()
medias_altura_peso_geral

In [ ]:
# Junção dos dois DataFrames
medias_comparadas = pd.concat([medias_altura_peso_medalhistas, medias_altura_peso_geral], 
                                      axis=1, 
                                      join='inner')

labels = pd.MultiIndex.from_product([['Medalhistas', 'Geral'], ['Height', 'Weight']])

medias_comparadas.set_axis(labels=labels, axis=1, inplace=True)

medias_comparadas

In [ ]:
def diferenca_percentual(linha, medida):
    if medida == 'altura':
        return (linha[0]/linha[2]-1)*100
    elif medida == 'peso':
        return (linha[1]/linha[3]-1)*100

dif_percent_alt = medias_comparadas.apply(diferenca_percentual, args=('altura', ), axis=1)

dif_percent_peso = medias_comparadas.apply(diferenca_percentual, args=('peso', ), axis=1)

In [ ]:
# Diferença da porcentagem entre os dois DataFrames
medias_comparadas = pd.concat([medias_comparadas, 
                              pd.concat([dif_percent_alt, dif_percent_peso], 
                                        axis=1, 
                                        keys=[('Diferenca %', 'Height'), ('Diferenca %', 'Weight')])], 
                             axis=1)
medias_comparadas

In [ ]:
# Verifica-se os esportes que possuem uma diferença expressiva entre os pesos dos Medalhistas X Participantes,
# fazendo-nos questionar qual seria o motivo de tal disparidade.

medias_comparadas['Diferenca %'][medias_comparadas['Diferenca %']['Weight'].abs()>10]

In [ ]:
# Verifica-se que não há uma diferença expressiva entre as alturas dos Medalhistas X Participantes

medias_comparadas['Diferenca %'][medias_comparadas['Diferenca %']['Height'].abs()>10]

Existe um detalhe importante passando batido até agora em nossa análise: as categorias esportivas costumam ser divididas por gênero justamente por conta de diferenças físicas entre homens e mulheres que poderiam influenciar no desempenho. Compare a altura e peso médios de atletas brasileiros por esporte segmentado por sexo.

In [ ]:
medias_altura_peso_F = df_medalhistas_br.groupby(['Sport', 'Sex'])[['Height', 'Weight']].\
            mean().xs('F', level=1)
            
medias_altura_peso_M = df_medalhistas_br.groupby(['Sport', 'Sex'])[['Height', 'Weight']].\
            mean().xs('M', level=1)

medias_comparadas_sexo = pd.concat([medias_altura_peso_F, medias_altura_peso_M], axis=1, join='inner')

labels = pd.MultiIndex.from_product([['Feminino', 'Masculino'], ['Height', 'Weight']])

medias_comparadas_sexo.set_axis(labels=labels, axis=1, inplace=True)

medias_comparadas_sexo

In [ ]:
dif_percent_alt_sexo = medias_comparadas_sexo.apply(diferenca_percentual, args=('altura', ), axis=1)
dif_percent_peso_sexo = medias_comparadas_sexo.apply(diferenca_percentual, args=('peso', ), axis=1)
medias_comparadas_sexo = pd.concat([medias_comparadas_sexo, 
                              pd.concat([dif_percent_alt_sexo, dif_percent_peso_sexo], 
                                        axis=1, 
                                        keys=[('Diferenca %', 'Height'), ('Diferenca %', 'Weight')])], 
                             axis=1)
medias_comparadas_sexo

In [ ]:
# Verifica-se os esportes que possuem uma diferença expressiva entre os pesos dos Feminino X Masculino,
# o que poderia ser usado em uma discussão sobre a inclusão de mulheres trans nas olimpíadas, por exemplo.

medias_comparadas_sexo['Diferenca %'][medias_comparadas_sexo['Diferenca %']['Weight'].abs()>10]

In [ ]:
# Verifica-se os esportes que possuem uma diferença expressiva entre as alturas dos Feminino X Masculino.

medias_comparadas_sexo['Diferenca %'][medias_comparadas_sexo['Diferenca %']['Height'].abs()>10]

Qual foi (ou quais foram) o maior medalhista brasileiro em quantidade total de medalhas?

In [ ]:
medalhas_por_atleta_br = df_medalhistas_br.groupby('Name')['Medal'].count()

maximo = medalhas_por_atleta_br.max()

brasileiros_mais_medalhas = medalhas_por_atleta_br[medalhas_por_atleta_br == maximo]

brasileiros_mais_medalhas

E o(s) maior(es) em quantidade de medalhas de ouro?

In [ ]:
ouros_por_atleta_br = df_medalhistas_br.groupby('Name')['Medal'].value_counts().loc[:, 'Gold']

maximo_br = ouros_por_atleta_br.max()

brasileiros_mais_ouros = ouros_por_atleta_br[ouros_por_atleta_br == maximo_br]

brasileiros_mais_ouros


Qual esporte rendeu mais medalhas de ouro para o Brasil?

**DICA:** tome muito cuidado nessa análise: cada **evento esportivo** rende 1 medalha. Por exemplo, quando a equipe de futebol vence, isso é considerado 1 medalha, mesmo tendo cerca de 20 atletas medalhistas na equipe. 

In [ ]:
df_medalhas_por_evento_br = df_medalhistas_br.drop_duplicates(subset=['Event','Year','Medal'])

df_ouros_por_evento_br = df_medalhas_por_evento_br[df_medalhas_por_evento_br['Medal'] == 'Gold']

ouros_por_esporte_br = df_ouros_por_evento_br.groupby('Sport').size()

maximo = ouros_por_esporte_br.max()

esportes_mais_ouros_br = ouros_por_esporte_br[ouros_por_esporte_br == maximo]

esportes_mais_ouros_br


E qual rendeu mais medalhas no total?

In [ ]:
medalhas_por_esporte_br = df_medalhas_por_evento_br.groupby('Sport').size()

maximo = medalhas_por_esporte_br.max()

esportes_mais_medalhados_br =  medalhas_por_esporte_br[medalhas_por_esporte_br == maximo]

esportes_mais_medalhados_br


Cada "categoria" dentro de um esporte é considerado um evento. Por exemplo, dentro de "atletismo", temos uma competição de 100m masculina, uma de 100m feminino, um revezamento 4 x 100m masculino, um revezamento 4 x 100m feminino, uma competição de 400m masculino, uma de 400m feminino, uma maratona masculina, uma maratona feminina, e assim sucessivamente.

Sabendo disso, qual evento esportivo mais rendeu medalhas de ouro para o Brasil? E total de medalhas?

In [ ]:
ouros_por_evento_br = df_ouros_por_evento_br.groupby('Event').size()

maximo = ouros_por_evento_br.max()

eventos_mais_ouros_br = ouros_por_evento_br[ouros_por_evento_br == maximo]

print('Evento esportivo que mais rendeu medalhas de ouro para o Brasil:')
eventos_mais_ouros_br


In [ ]:
medalhas_por_evento_br = df_medalhas_por_evento_br.groupby('Event').size()

maximo = medalhas_por_evento_br.max()

eventos_mais_medalhados_br = medalhas_por_evento_br[medalhas_por_evento_br == maximo]

print('Evento esportivo que mais rendeu medalhas para o Brasil:')
eventos_mais_medalhados_br


Para finalizar sobre o Brasil: obtenha o total de medalhas de ouro, prata, bronze e total por ano.

In [ ]:
medalhas_por_cor_br = df_medalhas_por_evento_br.groupby('Medal').size()

print('Total de medalhas do Brasil:')
medalhas_por_cor_br.sort_values()

In [ ]:
medalhas_por_ano_br = df_medalhas_por_evento_br.groupby('Year').size()
print('Total de medalhas do Brasil por ano:')
medalhas_por_ano_br


## 2. O mundo nos jogos de verão

Vamos agora analisar um pouquinho do que aconteceu nas Olimpíadas de verão em todo o mundo.

Retome o DataFrame original e descarte as informações sobre os jogos de inverno.

In [ ]:
df_verao = df[df['Season'] == 'Summer'].copy()

Obtenha a lista de todos os esportes já disputados nas olimpíadas de verão.

In [ ]:
df_verao['Sport'].value_counts().sort_index().index.tolist()

Obtenha a lista de todas as modalidades esportivas já disputadas nas olimpíadas de verão.

In [ ]:
df_verao['Event'].value_counts().sort_index().index.tolist()

Obtenha a lista de todos os países que já disputaram olimpíadas.

In [ ]:
# Observação: do modo como atualizamos a coluna 'Team' com um representante por 'NOC', 
# nessa lista não aparecem os paises componentes do Reino Unido: England, Gales, Escócia e Irlandas 
# além de domínios britânicos ultramarinos. Todos estão sob o nome 'Great Britain'

df_verao['Team'].sort_values().unique().tolist()

Qual atleta foi o maior medalhista (em medalhas totais) da história das olimpíadas de verão?

In [ ]:
df_medalhistas = df_verao[df_verao['Medal'] != 'No medal'].copy()

medalhas_por_atleta = df_medalhistas.groupby('Name')['Medal'].count()

maximo = medalhas_por_atleta.max()

maiores_medalhistas = medalhas_por_atleta[medalhas_por_atleta == maximo]
print('O maior medalhista da história das olimpíadas de verão:')
maiores_medalhistas

Qual atleta foi o maior medalhista de ouro da história das olimpíadas de verão?

In [ ]:
ouros_por_atleta = df_medalhistas[df_medalhistas['Medal'] == 'Gold'].groupby('Name').size()

maximo = ouros_por_atleta.max()

atletas_mais_ouros = ouros_por_atleta[ouros_por_atleta == maximo]

print('O maior medalhista de ouro da história das olimpíadas de verão:')
atletas_mais_ouros

Qual país foi o maior medalhista de ouro da história das olimpíadas de verão? Lembre-se da questão do evento esportivo, para não considerar múltiplas medalhas para um mesmo evento (ex: uma equipe de futebol fazendo parecer que mais de 20 medalhas foram distribuídas).

In [ ]:
df_medalhas_por_evento = df_medalhistas.drop_duplicates(subset=['Event','Year', 'Medal'])

df_ouros_por_pais = df_medalhas_por_evento[df_medalhas_por_evento['Medal'] == 'Gold']

ouros_por_pais = df_ouros_por_pais.groupby('Team').size()

maximo = ouros_por_pais.max()

paises_mais_ouros = ouros_por_pais[ouros_por_pais == maximo]

print('O maior país em medalhas de ouro da história das olimpíadas de verão:')
paises_mais_ouros

Qual país foi o maior medalhista em medalhas totais na história das olimpíadas de verão?

In [ ]:
medalhas_por_pais = df_medalhas_por_evento.groupby('Team').size()

maximo = medalhas_por_pais.max()

paises_mais_medalhados = medalhas_por_pais[medalhas_por_pais == maximo]

print('O maior país em medalhas da história das olimpíadas de verão:')
paises_mais_medalhados


Obtenha o total de medalhas de ouro, prata e total por edição das Olimpíadas de verão. Lembre-se da questão do evento esportivo.

In [ ]:
df_medalhas = df_medalhas_por_evento.groupby(['Year','Medal']).size()

medalhas = {'Gold':None,'Silver':None,'Bronze':None}

for x in medalhas.keys():
  medalhas[x] = df_medalhas.loc[:,x]
  
medalhas['Total'] = df_medalhas.groupby(level=0).sum()

df_medalhas_por_ano = pd.DataFrame(medalhas)
print('Total de medalhas por edição das Olimpíadas:')
df_medalhas_por_ano


## 3. Brasil vs Mundo

Para finalizar, vamos fazer algumas comparações entre Brasil e mundo. Qual o ranking do Brasil em cada edição das olimpíadas? Lembrando que o ranking é ordenado por medalhas de ouro.

In [ ]:
# Para considerar inclusive os países que não ganharam medalhas
df_por_evento = df_verao.drop_duplicates(subset=['Event','Year','Medal'])

In [ ]:
# Todos os países participantes
# df_qdro_med_ano = df_por_evento.groupby(['Year', 'Team', 'Medal']).size().reset_index()

# Apenas países com medalhas
df_qdro_med_ano = df_medalhas_por_evento.groupby(['Year', 'Team', 'Medal']).size().reset_index()

df_qdro_med_ano.rename(columns={0: 'Qtde'}, inplace=True)

df_qdro_med_ano['Gold'] = df_qdro_med_ano[df_qdro_med_ano['Medal']=='Gold']['Qtde']
df_qdro_med_ano['Silver'] = df_qdro_med_ano[df_qdro_med_ano['Medal']=='Silver']['Qtde']
df_qdro_med_ano['Bronze'] = df_qdro_med_ano[df_qdro_med_ano['Medal']=='Bronze']['Qtde']

df_qdro_med_ano = df_qdro_med_ano.sort_values(by = ['Year', 'Gold', 'Silver', 'Bronze'], 
                                     ascending=[True, False, False, False])

df_qdro_med_ano.drop(columns=['Medal','Qtde'],inplace = True)
df_qdro_med_ano.reset_index(drop=True)


In [ ]:
df_por_evento = df_verao.drop_duplicates(subset=['Event','Year','Medal']).copy()

valor_medalha = {'Gold': 1000000, 
                 'Silver': 1000, 
                 'Bronze': 1, 
                 'Sem medalhas': 0}

df_por_evento['valor_medalha'] = df_por_evento['Medal'].map(valor_medalha)

teste = df_por_evento.groupby(['Year', 'Team'])['valor_medalha'].sum().reset_index()

teste['Gold'] = teste['valor_medalha'] // 1_000_000
teste['Silver'] = (teste['valor_medalha'] % 1_000_000) // 1_000
teste['Bronze'] = teste['valor_medalha'] % 1_000

teste.sort_values(['Year', 'Gold', 'Silver', 'Bronze'], ascending=[True, False, False, False], inplace=True)
teste

In [ ]:
print('Ranking do Brasil nas edições das Olimpíadas que ele participou: \n')

for x in df_qdro_med_ano.groupby(['Year']):
    y = x[1].reset_index(drop=True)
    pos = y[y['Team'] == 'Brazil'].index + 1
    if len(pos) > 0: # Verifica se o Brasil participou das olimpíadas daquele ano
        print(f'{x[0]} - {pos[0]}°')

Compare o maior medalhista em ouros do Brasil com o maior medalhista em ouros do mundo.

In [ ]:
print('Brasileiros com mais ouros:')
print(brasileiros_mais_ouros)

print('\nAtletas com mais ouros:')
print(atletas_mais_ouros)


Compare o maior medalhista em total de medalhas do Brasil com o maior medalhista em total de medalhas do mundo.

In [ ]:
print('Brasileiros com mais medalhas:')
print(brasileiros_mais_medalhas)

print('\nAtletas com mais medalhas:')
print(maiores_medalhistas)

Compare o maior medalhista em ouros do Brasil com o maior medalhista do mundo no mesmo esporte.

In [ ]:
def printa_dicionario(dic):
  for k,v in dic.items():
    if type(k) != str:
        print(k[1],v)
        continue
    print(k,v)

In [ ]:
ouros_por_atleta_br = df_medalhistas_br.groupby(['Sport','Name'])['Medal'].value_counts().loc[:,:, 'Gold']

maximo_br = ouros_por_atleta_br.max()

brasileiros_mais_ouros = ouros_por_atleta_br[ouros_por_atleta_br == maximo_br]

esportes_comparacao = brasileiros_mais_ouros.index.get_level_values(0).unique().to_list()

ouros_por_atleta = df_medalhistas[df_medalhistas['Team'] != 'Brazil'].groupby(['Sport','Name'])['Medal'].value_counts().loc[esportes_comparacao,:, 'Gold'].droplevel(2)

In [ ]:
print('O maior medalhista em ouros do Brasil com o maior medalhista do mundo no mesmo esporte: \n')
for x in esportes_comparacao:
  maximo_x = ouros_por_atleta.groupby(level=0).max().loc[x]
  atletas_mais_ouros_x = ouros_por_atleta[(ouros_por_atleta.index.get_level_values(0) == x)&(ouros_por_atleta == maximo_x)].to_dict()

  print(x)

  print(f'\nBrasil')
  br = brasileiros_mais_ouros.loc[x,:].to_dict()
  printa_dicionario(br)

  print(f'\nMundo')
  printa_dicionario(atletas_mais_ouros_x)
  print('-'*50)

Compare o maior medalhista em total de medalhas do Brasil com o maior medalhista do mundo no mesmo esporte.

In [ ]:
medalhas_por_atleta_br = df_medalhistas_br.groupby(['Sport','Name'])['Medal'].size()

maximo_br = medalhas_por_atleta_br.max()

brasileiros_mais_medalhados = medalhas_por_atleta_br[medalhas_por_atleta_br == maximo_br]

esportes_comparacao = brasileiros_mais_medalhados.index.get_level_values(0).unique().to_list()

medalhas_por_atleta = df_medalhistas.groupby(['Sport','Name'])['Medal'].size().loc[esportes_comparacao,:]
# Caso não queira que os atletas brasileiros apareçam da tabela do mundo, basta substituir a linha anterior por:
# medalhas_por_atleta = df_medalhistas[df_medalhistas['Team'] != 'Brazil'].groupby(['Sport','Name'])['Medal'].size().loc[esportes_comparacao,:]

print('O maior medalhista do Brasil com o maior medalhista do mundo no mesmo esporte: \n')

for x in esportes_comparacao:
  maximo_x = medalhas_por_atleta.groupby(level=0).max().loc[x]
  atletas_mais_medalhas_x = medalhas_por_atleta[(medalhas_por_atleta.index.get_level_values(0) == x)&(medalhas_por_atleta == maximo_x)].to_dict()

  print(x)

  print(f'\nBrasil')
  br = brasileiros_mais_medalhados.loc[x,:].to_dict()
  printa_dicionario(br)

  print(f'\nMundo')
  printa_dicionario(atletas_mais_medalhas_x)
  print('-'*50)


Calcule o percentual de medalhas de ouro, prata e bronze que o Brasil ganhou em cada olimpíada.

In [ ]:
df_por_evento_br = df_por_evento[df_por_evento['Team'] == 'Brazil']
df_medalhas_br = df_por_evento_br[df_por_evento_br['Medal'] != 'No medal'].groupby(['Year','Medal']).size()

medalhas_br = {'Gold':None,'Silver':None,'Bronze':None}

for x in medalhas_br.keys():
  medalhas_br[x] = df_medalhas_br.loc[:,x].astype(np.int64)
  
medalhas_br['Total'] = df_medalhas_br.groupby(level=0).sum()

df_medalhas_por_ano_br = pd.DataFrame(medalhas_br).fillna(0)

print('Medalhas do Brasil nas Olimpíadas:')
df_medalhas_por_ano_br

In [ ]:
# Concatenação do DataFrame 'df_medalhas_por_ano_br' com 'df_medalhas_por_ano' .
df_comparativo_medalhas = pd.concat([df_medalhas_por_ano_br, df_medalhas_por_ano], 
                                            join='inner', 
                                            axis=1, 
                                            keys=['Brasil', 'Geral'])

df_comparativo_medalhas

In [ ]:
df_percent_medalhas_br = pd.DataFrame()

for x in df_comparativo_medalhas['Brasil'].columns:
  df_percent_medalhas_br[x] = round((df_comparativo_medalhas['Brasil'][x]/df_comparativo_medalhas['Geral'][x])*100, 2)

plt.figure(figsize=(17,10))
anos = df_percent_medalhas_br.index.to_list()

plt.bar(df_percent_medalhas_br.index-1,df_percent_medalhas_br['Bronze'],label = 'Bronze', color = 'Brown')
plt.bar(df_percent_medalhas_br.index,df_percent_medalhas_br['Silver'],label = 'Silver', color = 'Grey')
plt.bar(df_percent_medalhas_br.index+1,df_percent_medalhas_br['Gold'],label = 'Gold', color = 'Orange')

# plt.bar(df_percent_medalhas_br.index,df_percent_medalhas_br['Total'],label = 'Total', color = 'Blue')

plt.xlabel('Edições Olimpíadas')
plt.ylabel('% Medalhas')
plt.xticks(anos)


plt.title('Medalhas do Brasil por edição das Olimpíadas')
plt.legend(bbox_to_anchor=(1.01, 1.0))
plt.show()

In [ ]:
df_percent_medalhas_br